In [1]:
import wikipedia


def getWikiPage(link):
    try:
        composer = wikipedia.page(title=link, auto_suggest=False)
        return composer
    except:
        pass
    try:
        composer = wikipedia.page(title=link, auto_suggest=True)
        return composer
    except:
        return None

def addInformation(info_dict, name_set, tag_key, tag_value):
    for name in name_set:
        if name in info_dict:
            info_dict[name][tag_key] = tag_value
        else:
            info_dict[name] = {}
            info_dict[name][tag_key] = tag_value

composer_information = {}
links_set = set()

composers = set(getWikiPage("List_of_medieval_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'medieval')

composers = set(getWikiPage("List_of_Baroque_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'baroque')

composers = set(getWikiPage("List_of_Classical-era_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'classical')

composers = set(getWikiPage("List_of_Romantic-era_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'romantic')

composers = set(getWikiPage("List_of_20th-century_classical_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'modern')

composers = set(getWikiPage("List_of_21st-century_classical_composers").links)
links_set |= composers
addInformation(composer_information, composers, 'genre', 'contemporary')    

#links_set |= set(getWikiPage("List_of_medieval_composers").links)
#links_set |= set(getWikiPage("List_of_Renaissance_composers").links)
#links_set |= set(getWikiPage("List_of_Baroque_composers").links)
#links_set |= set(getWikiPage("List_of_Classical-era_composers").links)
#links_set |= set(getWikiPage("List_of_Romantic-era_composers").links)
#links_set |= set(getWikiPage("List_of_20th-century_classical_composers").links)
#links_set |= set(getWikiPage("List_of_21st-century_classical_composers").links)
links = list(links_set)


blacklist = ("romantic music", "list of")
relations = {}
index = 0
composer_count = 0
for link in links:
    try:
        if link.lower().startswith(blacklist):
            continue

        composer = getWikiPage(link)
        if composer == None:
            continue

        is_composer = False
        for category in composer.categories:
            if 'composer' in category.lower():
                is_composer = True
        if not is_composer:
            continue

        has_birthplace = False
        has_dob = False
        for c in composer.categories:
            if c.startswith('People from '):
                composer_information[link]['birthplace'] = c[len('People from '):]
                has_birthplace = True
            if c.endswith(' births'):
                composer_information[link]['dob'] = c[:-len(' births')]
                has_dob = True
        if not (has_birthplace and has_dob):
            del composer_information[link]
            continue

        relations[link] = composer.links
        composer_count += 1
        print("composer_count: " + str(composer_count))
    except:
        print("ERROR " + link + "------------------------------")
allComposers = relations.keys()


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900


KeyError: 'links'

In [ ]:
def filterComposerRelationsOnly(relationsOfComposer, allComposers):
    return [relation for relation in relationsOfComposer if relation in allComposers]

relations = {k: filterComposerRelationsOnly(relationsOfComposer, allComposers) for k, relationsOfComposer in relations.items()}


In [ ]:
file = open("followerFolloweeRomantic.txt","w") 
for followeeComposer,followedComposers in relations.items():
    for followedComposer in followedComposers:
        file.write(followeeComposer+","+followedComposer+"\n")  
file.close() 

In [ ]:
file = open("composer_information.txt","w")
for name, info in composer_information.items():
    if len(info) == 3:
        file.write(name + ',' + info['dob'] + ',' + info['birthplace'] + ',' + info['genre'] + '\n')
file.close()